# video.summrazer.ipynb

Download the video from Youtube
First of all, we need a way to download the video from youtube. Actually, we don’t need the whole video but only the audio. So we will extract the audio from the video and download only that.

The library I used to interoperate with youtube is youtube_dl which you can learn more about on GitHub.

So we install the library with pip and download the audio from youtube in the following way.

In [4]:
from moviepy.editor import VideoFileClip

# Specify the path to your local video file
local_video_path = "bigo.mp4"

# Load the video clip
video_clip = VideoFileClip(local_video_path)

# Extract audio from the video clip
audio_clip = video_clip.audio

# Specify the output audio file name
output_audio_path = "output_audio.wav"

# Write the audio to a WAV file
audio_clip.write_audiofile(output_audio_path)

# Close the video clip to release resources
video_clip.close()

print(f"Audio extracted and saved to {output_audio_path}")


MoviePy - Writing audio in output_audio.wav


MoviePy - Done.
Audio extracted and saved to output_audio.wav


Note that in line 8 we chose to download the audio in wav format, but mp3 or others are also fine if you prefer.
In line 15, on the other hand, you must enter the link to the video you want.

Listen to the Audio
Did we download the audio correctly? Let’s check by resenting the audio directly from the notebook.

In [1]:
!pip install pocketsphinx

import speech_recognition as sr

# Define the path to your WAV file
wav_file = "output_audio.wav"

# Initialize the recognizer
recognizer = sr.Recognizer()

# Load the audio file
with sr.AudioFile(wav_file) as source:
    # Listen for the data (load audio to memory)
    audio_data = recognizer.record(source)
    
    # Recognize speech using the Sphinx engine (offline)
    try:
        text = recognizer.recognize_sphinx(audio_data)
        print("Extracted text:", text)
    except sr.UnknownValueError:
        print("Sphinx could not understand the audio")
    except sr.RequestError as e:
        print("Sphinx error; {0}".format(e))


![alt text](image.png)

Audio to Text
The next step is to convert the audio file into text hoping to get a low word error rate. This will be useful since we can then run a summarization NLP algorithm directly on the text.

You can read more about the model we will use for text conversion to text here.

In [11]:
%%capture
!pip install transformers
from transformers import pipeline

model = "facebook/wav2vec2-large-960h-lv60-self" #speech to text

#speech to text
pipe = pipeline(model = model)
text = pipe('output_audio.wav', chunk_length_s=10) 

#save text
text_file = open("original_text.txt", "w")
n = text_file.write(text["text"])
text_file.close()

#read article
text_article = open("original_text.txt", "r").read()
print(len(text_article.split()))
text_article

Text Summarization
Now what’s left for us to do is to take the text we extracted from the video and summarize it.
There are hundreds of summarization models, all you have to do is go to hugging face filter on the summarization button and choose the one that best suits your case.

For this project, I will use the google/pegasus-xsum model. You can read the details of this model here (in some future articles I will also go on to explain the theory behind these summarization algorithms).

Using these pre-trained models found on HugginFace is really simple, look at I use summarization in a few lines of code.

In [3]:
%pip install sentencepiece

from transformers import pipeline

# Define the filename here
filename = "original_text.txt"

# Read the text from the file
with open(filename, 'r') as file:
    text_article = file.read()

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
tokenizer_kwargs = {'truncation':True,'max_length':512}
text_summerization = summarizer(text_article, min_length=30, do_sample=False,**tokenizer_kwargs)

print(text_summerization)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


config.json: 100%|██████████| 1.58k/1.58k [00:00<00:00, 7.54MB/s]
model.safetensors: 100%|██████████| 1.63G/1.63G [00:11<00:00, 138MB/s] 
generation_config.json: 100%|██████████| 363/363 [00:00<00:00, 1.92MB/s]
vocab.json: 100%|██████████| 899k/899k [00:00<00:00, 1.13MB/s]
merges.txt: 100%|██████████| 456k/456k [00:00<00:00, 30.6MB/s]
tokenizer.json: 100%|██████████| 1.36M/1.36M [00:00<00:00, 30.2MB/s]


[{'summary_text': 'The BEGO NOTATION IS USED IN ACADEMICS TO DESCRIBE WRONG TIMES. In the NURSERY PEOPLE TEND TO USE BIGO TO UNDERSTAND DIFFERENT NOTATIONS THAT ARE USED TO MEASURE THE PERFORMANCE OF ALGRIDIM. We will look at real life examples to see how different cars perform in different conditions. We are going to use the MARKER MARKER MOVES TO THE RIGHT IN EACH EACH STEP.'}]


![alt text](image-1.png)